In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD, Adam
from keras import backend as K
import pandas as pd
import numpy as np
import ipdb

/home/sean/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 读入数据
由于数据集较大，我们这里采用将其切成一个个ｃｈｕｎｋ来读入，否则内存会全部背耗光

In [2]:
def loadData():
    reader_1 = pd.read_csv('train.csv', chunksize=1000)
    y_train = []
    x_train = []
    for chunk in reader_1:
        y_train += list(chunk.label)
        for i in chunk.feature:
            i = list(map(int, i.split(' ')))
            x_train += [i]
    y_train = np.array(y_train)
    x_train = np.array(x_train)

    reader_2 = pd.read_csv('test.csv', chunksize=1000)
    x_test = []
    for chunk in reader_2:
        for i in chunk.feature:
            i = list(map(int, i.split(' ')))
            x_test += [i]
    x_test = np.array(x_test)
    return x_train, y_train, x_test

In [3]:
x_train, y_train, x_test = loadData()

In [4]:
x_train = x_train.reshape(x_train.shape[0], 48, 48)
x_test = x_test.reshape(x_test.shape[0], 48, 48)

# 玄学调参

In [5]:
img_rows = 48
img_cols = 48
num_classes = 7

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)

x_train shape: (28709, 48, 48, 1)
28709 train samples
7178 test samples


In [ ]:
model = Sequential()
model.add(Conv2D(25, kernel_size=(3, 3),activation='relu', input_shape=input_shape))
model.add(Conv2D(50, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(output_dim = 100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_dim = num_classes, activation='softmax'))
    
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=12)
score = model.evaluate(x_train, y_train, batch_size=10000)
print('Train loss', score[0])
print('Train accuracy', score[1])
yhat = model.predict(x_test, batch_size=100)
yhat = np.argmax(yhat, axis=1)

/home/sean/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
  import sys
/home/sean/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=7)`
  if __name__ == '__main__':


Epoch 1/12
28709/28709 [==============================] - 151s 5ms/step - loss: 1.7119 - acc: 0.3151
Epoch 2/12
28709/28709 [==============================] - 138s 5ms/step - loss: 1.5034 - acc: 0.4254
Epoch 3/12
28709/28709 [==============================] - 137s 5ms/step - loss: 1.3735 - acc: 0.4792
Epoch 4/12
28709/28709 [==============================] - 137s 5ms/step - loss: 1.2749 - acc: 0.5190
Epoch 5/12
28709/28709 [==============================] - 137s 5ms/step - loss: 1.1782 - acc: 0.5592
Epoch 6/12
28709/28709 [==============================] - 168s 6ms/step - loss: 1.0728 - acc: 0.6028
Epoch 7/12
28709/28709 [==============================] - 168s 6ms/step - loss: 0.9395 - acc: 0.6579
Epoch 8/12
28709/28709 [==============================] - 143s 5ms/step - loss: 0.7843 - acc: 0.7202
Epoch 9/12
28709/28709 [==============================] - 139s 5ms/step - loss: 0.6121 - acc: 0.7884
Epoch 10/12
28709/28709 [==============================] - 138s 5ms/step - loss: 0.4390 - a

In [ ]:
score = model.evaluate(x_train, y_train, batch_size=10000)
print('Train loss', score[0])
print('Train accuracy', score[1])
yhat = model.predict(x_test, batch_size=100)
yhat = np.argmax(yhat, axis=1)
result = pd.DataFrame(yhat)
result.to_csv('submission.csv')